### 2. Convolution Arithmetic

참고! kernel은 홀수 값으로 보통 사용.

* unit padding. 패딩을 주지 않을 때 (프레임워크에서 valid=True 라는 옵션.)   
* Half- padding (= Same Padding). input size와 output size를 똑같이 만들어주는 패딩 (same padding 이라는 옵션)   
* Full padding : input size < output size  



### 3. Pooling, Transposed Convolution Layer

#### Pooling

1. max pooling
2. average pooling

#### transposed convolution layer

아웃풋 값을 원래 이미지로 복원하는 역할
이미지에서는 upsampling이라고 부름

왜 transposed convolution을 쓸가?

* interpolation ; 픽셀과 픽셀 사이에 중간값을 추가하여 이미지 복원.  
but, 이미지 사이즈가 커지면 연산이 늘어나 적합하지 않음

* direct convolution : 역연산과 방법이 비슷. 
but, 0이 많이 추가되어 비효율적일 수 있다.



장점 : kernal, input, output 의 input, ouput을 flatten화, kernal은 이 형상에 맞춰서 0으로 패딩 된다.


쓰임 : decoder, 특히 GAN 에서 많이 쓰임.

---

# 1. Box and Whisker Plot (= Box Plot)

값들을 통계적 값으로 표현.

![](https://miro.medium.com/max/18000/1*2c21SkzJMf3frPXPAR_gZA.png)

### mean vs median

mean : 평균  
median : 중앙값. 아웃라이어의 영향을 덜 받는다.  

### Quartile 

quar : 1/4의 의미  

3개의 quartile을 가짐. 


IQR : Inter quartile range. Q1 ~ Q3. (하위 25% ~ 상위 25%까지의 보편적인 중앙값)

IQR이 줄어들면 분산이 줄어든다고 해석 가능.

---

* 아웃라이어의 기준을 어디로 해야할까?

1.5 * IQR : 아웃라이어 기준. 아래로 위로.
1.5 IQR까지 그린 것이 whisker. 

* 최대값, 최소값까지 whisker 하는 경우도 있긴 함.



# 2. HOG (Histogram of Oriented Gradients)

CNN에서 앞 부분은 feature extraction, 뒷 부분은 classification.  
CNN이 나오기 전 어떻게 image를 다루었을까? 어떻게 feature를 extract 했을지 고민해보자! 

1. Feature Detector : 단순히 location 기준  
2. Feature Descripter : 이미지 -> 벡터로.  

* 질문 ! 좋은 피처란 ?

**서로 다른 클래스 간 차이가 명확한 것**

즉, ideal feature extractor : classifier가 일을 안해도 될 정도  
다른 클래스를 잘 구분하려면 노이즈가 적어야하고, 클래스끼리 차이가 커야 한다.  
Good feature should be informative(= 클래스 간 차이가 큰 것), invariant to noise.  

## HOG
 
딥러닝이 나오기 전 featur descripter 중 하나. 한번 hand made feature 벡터들로 해보자!  
클래식한 방법이지만 지금까지도 많이 사용하는 방법!  

### Step 1. Edge Detection (Gradient Image)

딥러닝에서는 Convolution으로 edge detect 했었다면, 이전에는 어떻게 했을까?

이미지를 x축에 대한 미분, y축에 대한 미분.

**1. magnitude**

x축에 대한 미분, y축에 대한 미분을 각 각 제곱하여 더한 다음 루트 씌우면 -> 피타고라스 정리에 의해 길이가 나옴. 화살표 : magnitude  
차이가 크면 화살표가 길 것. (예를 들어 사진 상에서 어두웠다가 갑자기 밝아지면 화살표 길이가 길어질 것.)  

**2. angle**

arctan (y에 대한 미분 / x에 대한 미분) : angle


이 magnitude와 angle을 함께 고려해서 그린 화살표 이미지 : gradient image

Prewitt, Sobel 기법 등 있음.

---

### Step 2. Histogram of Gradients

cell. 크기는 8 X 8로 주는 것이 일반적인데, 

0도 ~ 180도 -> 0-20 ,20-40 , .. 총 9개 (bin이라고 함)

단순 count 가 들어온 것이 아니라 angle과 magnitude를 함께 고려하여 weight를 주어 히스토그램을 그린다.  
-> 단순히 개수를 count하면 noise가 커질 수 있기 때문.

가령, 어떤 cell의 angle (direction) 이 10이면 0 / 20 / 40 ..의 bin 중 딱 0곽 20 사이일 경우, 해당되는 cell의 magnitude가 4이면 딱 절반씩 잘라서 2씩 0과 20에 들어가 히스토그램을 그린다.

![](https://i.imgur.com/EbXbVQl.png)
![](https://www.learnopencv.com/wp-content/uploads/2016/12/hog-cell-gradients.png)

### Step 3. Block Normalization

Block 등장 . 2X2의 cell들 = Block. 블락 기준 정규화.  
정규화 하는 이유 : noise에 둔감해지기 위해 정규화 한 것. 즉 contraction, distortion에 robust해짐.

벡터의 정규화 : unit 벡터로 만들기. 자기 자신으로 나누어 준 것 (3,4)이면 $root(3^2 + 4^2)$로 각 각 나누어 주기.

0~180까지 9개의 값. 4개의 cell이므로 한 블록에는 36개의 값.
flatten화 하여 정규화 한 값이 해당 블록의 hog feature

![](https://i.stack.imgur.com/HK9Oq.png)



이미지의 경향성.

MNIST를 예시로 : 28 * 28 이미지에서 8 * 8 cell 이 아니라 4 * 4로 잡으면, 총 7 * 7 사이즈의 hog feature가 나옴.

#### 과제! HOG EXTRACTOR 만들기

이번 프로젝트 ! 클래식한 feature extractor + MLP. 직접 feature extractor 만들면 어떻게 될지. informative 한 벡터를 넣어주기.
주의! normalize 한 값을 새로운 matrix에 넣어주어야 함. 원래 자기 자신의 이미지 값에 넣으면 안됨!! (왜곡될 수 있음)